In [34]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler


scalar1 = MinMaxScaler()
scalar2 = MinMaxScaler()

X_train_score = pd.read_csv('trainX_score.csv')  
y_train_score = pd.read_csv('trainY_score.csv')

y_train_score = y_train_score.replace({"Over": True, "Under": False})

X_train_score = X_train_score.to_numpy(dtype=np.float32)
y_train_score = y_train_score.to_numpy(dtype=np.float32)

X_train_spread = pd.read_csv('trainX_spread.csv')
y_train_spread = pd.read_csv('trainY_spread.csv')

y_train_spread = y_train_spread.replace({"Cover": True, "Did Not Cover": False})

X_train_spread = X_train_spread.to_numpy(dtype=np.float32)
y_train_spread = y_train_spread.to_numpy(dtype=np.float32)

X_train_score = scalar1.fit_transform(X_train_score)
X_train_spread = scalar2.fit_transform(X_train_spread)


X_test_score = pd.read_csv('testX_score.csv')
y_test_score = pd.read_csv('testY_score.csv')

y_test_score = y_test_score.replace({"Over": True, "Under": False})

X_test_score = X_test_score.to_numpy(dtype=np.float32)
y_test_score = y_test_score.to_numpy(dtype=np.float32)

X_test_spread = pd.read_csv('testX_spread.csv')
y_test_spread = pd.read_csv('testY_spread.csv')

y_test_spread = y_test_spread.replace({"Cover": True, "Did Not Cover": False})

X_test_spread = X_test_spread.to_numpy(dtype=np.float32)
y_test_spread = y_test_spread.to_numpy(dtype=np.float32)

X_test_score = scalar1.transform(X_test_score)
X_test_spread = scalar2.transform(X_test_spread)

def toSequences(seq_size, columns, X, y):
    X_seq = []
    y_seq = []
    for i in range(len(X) - seq_size):
        X_seq.append(X[i:i+seq_size])
        y_seq.append(y[i+seq_size-1]) 
    return torch.tensor(X_seq, dtype=torch.float32).view(-1, seq_size, columns), torch.tensor(y_seq, dtype=torch.float32).view(-1, 1)

X_train_score, y_train_score = toSequences(20, X_train_score.shape[1], X_train_score, y_train_score)
X_test_score, y_test_score = toSequences(20, X_test_score.shape[1], X_test_score, y_test_score)

X_train_spread, y_train_spread = toSequences(20, X_train_spread.shape[1], X_train_spread, y_train_spread)
X_test_spread, y_test_spread = toSequences(20, X_test_spread.shape[1], X_test_spread, y_test_spread)

C:\Users\Twk23\AppData\Local\Temp\ipykernel_5676\3747382305.py:16: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_train_score = y_train_score.replace({"Over": True, "Under": False})
C:\Users\Twk23\AppData\Local\Temp\ipykernel_5676\3747382305.py:24: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_train_spread = y_train_spread.replace({"Cover": True, "Did Not Cover": False})
C:\Users\Twk23\AppData\Local\Temp\ipykernel_5676\3747382305.py:36: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a futu

In [35]:
print(X_train_score.shape, y_train_score.shape)
print(X_test_score.shape, y_test_score.shape)
print(X_train_spread.shape, y_train_spread.shape)
print(X_test_spread.shape, y_test_spread.shape)

torch.Size([2608, 20, 19]) torch.Size([2608, 1])
torch.Size([855, 20, 19]) torch.Size([855, 1])
torch.Size([2599, 20, 24]) torch.Size([2599, 1])
torch.Size([852, 20, 24]) torch.Size([852, 1])


In [38]:
class TransformerTimeSeriesModel(nn.Module):
    def __init__(self, input_size, d_model, nhead, num_encoder_layers, dim_feedforward, output_size, dropout=0.1):
        super(TransformerTimeSeriesModel, self).__init__()
        
        self.input_fc = nn.Linear(input_size, d_model)
        self.positional_encoding = nn.Parameter(
            torch.zeros(1000, d_model) 
        )
        # Transformer Encoder
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout),
            num_layers=num_encoder_layers
        )
        # Fully connected output layer
        self.output_fc = nn.Linear(d_model, output_size)
        # Optional dropout to avoid overfitting
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x):
        x = self.input_fc(x)  
        seq_len = x.size(1)
        x = x + self.positional_encoding[:seq_len, :].unsqueeze(0)
        x = x.permute(1, 0, 2)
        x = self.transformer_encoder(x)  
        x = x.mean(dim=0) 
        x = self.dropout(x)
        x = self.output_fc(x)
        
        return x

train_dataset_score = TensorDataset(X_train_score, y_train_score)
train_loader_score = DataLoader(train_dataset_score, batch_size=32, shuffle=True)

test_dataset_score = TensorDataset(X_test_score, y_test_score)
test_loader_score = DataLoader(test_dataset_score, batch_size=32, shuffle=False)

train_dataset_spread = TensorDataset(X_train_spread, y_train_spread)
train_loader_spread = DataLoader(train_dataset_spread, batch_size=32, shuffle=True)

test_dataset_spread = TensorDataset(X_test_spread, y_test_spread)
test_loader_spread = DataLoader(test_dataset_spread, batch_size=32, shuffle=False)

input_size = X_train_score.shape[2]
d_model = 64           
nhead = 4               
num_encoder_layers = 3  
dim_feedforward = 128    
output_size = 1         
print(input_size)
over_under_model = TransformerTimeSeriesModel(input_size, d_model, nhead, num_encoder_layers, dim_feedforward, output_size)

input_size = X_train_spread.shape[2]      
d_model = 64           
nhead = 4               
num_encoder_layers = 3  
dim_feedforward = 128    
output_size = 1 
print(input_size)
spread_model = TransformerTimeSeriesModel(input_size, d_model, nhead, num_encoder_layers, dim_feedforward, output_size)

def train_model(model, train_loader, imbalanced_ratio, num_epochs=300):
    pos_weight = torch.tensor([imbalanced_ratio])
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    model.train()

    for epoch in range(num_epochs):
        running_loss = 0.0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()  # Zero gradients

            # Forward pass
            outputs = model(X_batch)
            loss = criterion(outputs.squeeze(), y_batch.squeeze())

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * X_batch.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

num_positive = (y_train_score == 1).sum().item()
num_negative = (y_train_score == 0).sum().item()
imbalanced_ratio = num_negative / num_positive

train_model(over_under_model, train_loader_score, imbalanced_ratio)

num_positive = (y_train_spread == 1).sum().item()
num_negative = (y_train_spread == 0).sum().item()
imbalanced_ratio = num_negative / num_positive

train_model(spread_model, train_loader_spread, imbalanced_ratio)

19
24


c:\Users\Twk23\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/300, Loss: 0.7239
Epoch 2/300, Loss: 0.7097
Epoch 3/300, Loss: 0.7118
Epoch 4/300, Loss: 0.7096
Epoch 5/300, Loss: 0.7095
Epoch 6/300, Loss: 0.7084
Epoch 7/300, Loss: 0.6903
Epoch 8/300, Loss: 0.6715
Epoch 9/300, Loss: 0.6559
Epoch 10/300, Loss: 0.6485
Epoch 11/300, Loss: 0.6496
Epoch 12/300, Loss: 0.6388
Epoch 13/300, Loss: 0.6308
Epoch 14/300, Loss: 0.6244
Epoch 15/300, Loss: 0.6141
Epoch 16/300, Loss: 0.6215
Epoch 17/300, Loss: 0.6156
Epoch 18/300, Loss: 0.6056
Epoch 19/300, Loss: 0.6201
Epoch 20/300, Loss: 0.5961
Epoch 21/300, Loss: 0.6020
Epoch 22/300, Loss: 0.5916
Epoch 23/300, Loss: 0.5863
Epoch 24/300, Loss: 0.5830
Epoch 25/300, Loss: 0.5689
Epoch 26/300, Loss: 0.5692
Epoch 27/300, Loss: 0.5701
Epoch 28/300, Loss: 0.5481
Epoch 29/300, Loss: 0.5635
Epoch 30/300, Loss: 0.5531
Epoch 31/300, Loss: 0.5369
Epoch 32/300, Loss: 0.5418
Epoch 33/300, Loss: 0.5286
Epoch 34/300, Loss: 0.5191
Epoch 35/300, Loss: 0.5112
Epoch 36/300, Loss: 0.5096
Epoch 37/300, Loss: 0.5089
Epoch 38/3

In [39]:
from sklearn.metrics import classification_report, confusion_matrix

over_under_model.eval()

with torch.no_grad():
    predictions = torch.round(torch.sigmoid(over_under_model(X_test_score)))

# print(predictions)
predictions = predictions.cpu().numpy().flatten()
report = classification_report(y_test_score, predictions)
print("Over/Under Classification Report:")
print(report)

conf_matrix = confusion_matrix(y_test_score, predictions)
print(f"Confusion Matrix:\n{conf_matrix}")

Over/Under Classification Report:
              precision    recall  f1-score   support

         0.0       0.60      0.61      0.60       470
         1.0       0.51      0.51      0.51       385

    accuracy                           0.56       855
   macro avg       0.56      0.56      0.56       855
weighted avg       0.56      0.56      0.56       855

Confusion Matrix:
[[285 185]
 [189 196]]


In [41]:
spread_model.eval()

with torch.no_grad():
    predictions = torch.round(torch.sigmoid(spread_model(X_test_spread)))

# print(predictions)
predictions = predictions.cpu().numpy().flatten()
report = classification_report(y_test_spread, predictions)
print("Spread Classification Report:")
print(report)

conf_matrix = confusion_matrix(y_test_spread, predictions)
print(f"Confusion Matrix:\n{conf_matrix}")

Spread Classification Report:
              precision    recall  f1-score   support

         0.0       0.62      0.62      0.62       448
         1.0       0.58      0.58      0.58       404

    accuracy                           0.60       852
   macro avg       0.60      0.60      0.60       852
weighted avg       0.60      0.60      0.60       852

Confusion Matrix:
[[276 172]
 [170 234]]
